In [1]:
# Use this to play around with game feature generator features 
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
import os
from tqdm import tqdm
from copy import deepcopy
from data_processing.utils.download_functions import *

In [4]:
os.chdir('esports-data')
os.listdir()

['games',
 'leagues.json',
 'mapping_data.json',
 'players.json',
 'teams.json',
 'tournaments.json']

In [5]:
with open("mapping_data.json", "r") as json_file:
   mappings_data = json.load(json_file)

mappings = {
   esports_game["esportsGameId"]: esports_game for esports_game in mappings_data
}

game_id = '110303581088331459'
game_mapping_data = mappings[game_id]
platform_game_id = game_mapping_data['platformGameId']
with open(f"games/{platform_game_id}.json", "r") as json_file:
    game_data = json.load(json_file)


In [8]:
from datetime import datetime
import pandas as pd
import numpy as np


class GameFeaturesGenerator:
    TIME_FORMAT = '%Y-%m-%dT%H:%M:%S.%fZ'

    @staticmethod
    def get_event_time(start_time, event_time):
        return (datetime.strptime(event_time, GameFeaturesGenerator.TIME_FORMAT) - start_time).total_seconds()

    @staticmethod
    def flip_team_id(team_id):
        """
        Flips team_id from 100 to 200 and vice versa. Needed for when processing turret fall events because we want to update the feature for the team that lost the turret
        """
        if team_id == 100:
            return 200
        elif team_id == 200:
            return 100
        else:
            print("Error: team_id is not 100 or 200")

    def __init__(self, game_data, mapping_data):
        self.game_data = game_data
        self.game_start_time = datetime.strptime(game_data[0]['eventTime'], GameFeaturesGenerator.TIME_FORMAT)
        self.team_id_mapping = {"100": mapping_data['teamMapping']['100'], "200": mapping_data['teamMapping']['200']}
        self.esports_game_id = mapping_data['esportsGameId']  # Looks like '110310652412257228'
        self.platform_game_id = mapping_data['platformGameId']  # Looks lik {'esportsGameId': '110310652412257228'
        # Create flags for first events (e.g. first turret kill, first dragon kill, etc.). These can only happen once (for both teams)
        self.first_herald_flag = True
        self.first_dragon_flag = True
        self.first_baron_flag = True
        self.first_turret_flag = True
        self.first_inhibitor_flag = True
        self.first_kill_flag = True

        # Create features for each team
        # EPIC MONSTER EVENT FEATURES
        self.IMPORTANT_MONSTER_TYPES = ["riftHerald", "dragon", "baron"]
        epic_monster_kill_event_features = [
            ["first_" + monster_type + "_ind", "first_" + monster_type + "_time", "num_" + monster_type] for
            monster_type
            in self.IMPORTANT_MONSTER_TYPES]
        epic_monster_kill_event_features = [item for sublist in epic_monster_kill_event_features for item in
                                            sublist]  # Flatten list of lists

        # BUILDING DESTROYED EVENT FEATURES
        self.IMPORTANT_BUILDING_TYPES = ["turret", "inhibitor"]
        building_destroyed_event_features = [
            ["first_" + building_type + "_ind", "first_" + building_type + "_time", "num_" + building_type] for
            building_type
            in self.IMPORTANT_BUILDING_TYPES]
        building_destroyed_event_features = [item for sublist in building_destroyed_event_features for item in
                                             sublist]  # Flatten list of lists

        # CHAMPION KILL EVENT FEATURES
        champion_kill_event_features = ["first_kill_ind", "first_kill_time", "num_kills"]

        # GAME METADATA FEATURES
        # TODO: Also need a feature for whether this was a domestic or international game (e.g. LCS vs. Worlds)
        game_metadata_features = ["game_end_time"]

        all_features = (["platformGameId", "esportsGameId", "team_id", "start_time", "outcome"] +
                        epic_monster_kill_event_features + building_destroyed_event_features + champion_kill_event_features + game_metadata_features)
        self.team_features = {"100": {feature: np.nan for feature in all_features},
                              "200": {feature: np.nan for feature in all_features}}
        # Set all "num_" features to 0
        for team_id in ["100", "200"]:
            for feature in all_features:
                if feature.startswith("num_"):
                    self.team_features[team_id][feature] = 0

    def process_epic_monster_kill_event(self, event, team_features):
        monster_type = event['monsterType']
        if monster_type in self.IMPORTANT_MONSTER_TYPES:
            event_time = event['eventTime']
            time_of_kill = GameFeaturesGenerator.get_event_time(self.game_start_time, event_time)
            killer_team_id = str(event['killerTeamID'])
            victim_team_id = str(GameFeaturesGenerator.flip_team_id(event['killerTeamID']))
            if self.first_herald_flag:
                team_features[killer_team_id]['first_riftHerald_ind'] = 1
                team_features[victim_team_id]['first_riftHerald_ind'] = 0
                team_features[killer_team_id]['first_riftHerald_time'] = time_of_kill
                self.first_herald_flag = False
            if self.first_dragon_flag:
                team_features[killer_team_id]['first_dragon_ind'] = 1
                team_features[victim_team_id]['first_dragon_ind'] = 0
                team_features[killer_team_id]['first_dragon_time'] = time_of_kill
                self.first_dragon_flag = False
            if self.first_baron_flag:
                team_features[killer_team_id]['first_baron_ind'] = 1
                team_features[victim_team_id]['first_baron_ind'] = 0
                team_features[killer_team_id]['first_baron_time'] = time_of_kill
                self.first_baron_flag = False
            team_features[killer_team_id]['num_' + monster_type] += 1

    def process_building_destroyed_event(self, event, team_features):
        building_type = event['buildingType']
        if building_type in self.IMPORTANT_BUILDING_TYPES:
            event_time = event['eventTime']
            time_of_event = GameFeaturesGenerator.get_event_time(self.game_start_time, event_time)
            team_id = str(GameFeaturesGenerator.flip_team_id(event['teamID']))
            victim_team_id = str(event['teamID'])
            if self.first_turret_flag:
                team_features[team_id]['first_turret_ind'] = 1
                team_features[victim_team_id]['first_turret_ind'] = 0
                team_features[team_id]['first_turret_time'] = time_of_event
                self.first_turret_flag = False
            if self.first_inhibitor_flag:
                team_features[team_id]['first_inhibitor_ind'] = 1
                team_features[victim_team_id]['first_inhibitor_ind'] = 0
                team_features[team_id]['first_inhibitor_time'] = time_of_event
                self.first_inhibitor_flag = False
            team_features[team_id]['num_' + building_type] += 1

    def process_champion_kill_event(self, event, team_features):
        event_time = event['eventTime']
        time_of_event = GameFeaturesGenerator.get_event_time(self.game_start_time, event_time)
        killer_team_id = str(event['killerTeamID'])
        victim_team_id = str(GameFeaturesGenerator.flip_team_id(event['killerTeamID']))
        if self.first_kill_flag:
            team_features[killer_team_id]['first_kill_ind'] = 1
            team_features[victim_team_id]['first_kill_ind'] = 0
            team_features[killer_team_id]['first_kill_time'] = time_of_event
            self.first_kill_flag = False
        team_features[killer_team_id]['num_kills'] += 1

    def process_game_end_event(self, event, team_features):
        # We only call this method for the final event, so if for some reason there is no game_end event then discard the data
        if event['eventType'] == "game_end":
            event_time = event['eventTime']
            time_of_event = GameFeaturesGenerator.get_event_time(self.game_start_time, event_time)
            team_features['100']['game_end_time'] = time_of_event
            team_features['200']['game_end_time'] = time_of_event
            winning_team = event['winningTeam']
            losing_team = GameFeaturesGenerator.flip_team_id(winning_team)
            team_features[str(winning_team)]['outcome'] = 1
            team_features[str(losing_team)]['outcome'] = 0
        else:
            raise Exception("Error: event is not a game_end event")

    def process_game(self):
        """
        Loops through all events in the game and updates the team features as they occur
        :return: a pandas dataframe with the features for each team and label "outcome"
        """
        for event in self.game_data:
            if event['eventType'] == "epic_monster_kill":
                self.process_epic_monster_kill_event(event, self.team_features)
            elif event['eventType'] == "building_destroyed":
                self.process_building_destroyed_event(event, self.team_features)
            elif event['eventType'] == "champion_kill":
                self.process_champion_kill_event(event, self.team_features)
            elif event['eventType'] == "game_end":
                self.process_game_end_event(event, self.team_features)
            else:
                continue

        # Now we have all the datapoints, we can create a dataframe with the team ID, label, start_time, and features
        # First assign the rest of the metadata
        rows = []
        for team_id in ["100", "200"]:
            # Assign metadata (same for both teams)
            self.team_features[team_id]['platformGameId'] = self.platform_game_id
            self.team_features[team_id]['esportsGameId'] = self.esports_game_id
            self.team_features[team_id]['start_time'] = self.game_start_time

            # Assing team-specific data
            self.team_features[team_id]['team_id'] = self.team_id_mapping[team_id]
            rows.append(pd.DataFrame.from_dict(self.team_features[team_id], orient='index').transpose())
        return pd.concat(rows, ignore_index=True)


In [127]:
# Notes: 
# Participants 1-5 are on team 100 and 6-10 are on team 200
# participant_mapping = {
#     '1': '100_top',
#     '2': '100_jungle',
#     '3': '100_mid',
#     '4': '100_bot',
#     '5': '100_support',
#     '6': '200_top',
#     '7': '200_jungle',
#     '8': '200_mid',
#     '9': '200_bot',
#     '10': '200_support'
# }



In [126]:
participant_data

{'ability4CooldownRemaining': 0,
 'magicPenetrationPercent': 0,
 'alive': True,
 'primaryAbilityResource': 955,
 'spellVamp': 0,
 'participantID': 10,
 'lifeSteal': 0,
 'primaryAbilityResourceRegen': 91,
 'ability1CooldownRemaining': 1.8475341796875,
 'XPForNextLevel': 7300,
 'summonerSpell2CooldownRemaining': 0,
 'healthMax': 1254,
 'magicResist': 36,
 'ability3CooldownRemaining': 0,
 'ability3Level': 4,
 'ability2CooldownRemaining': 0,
 'summonerSpell1Name': 'SummonerHeal',
 'teamID': 200,
 'currentGold': 616,
 'healthRegen': 18,
 'ability4Level': 1,
 'level': 10,
 'ability1Level': 5,
 'armorPenetration': 0,
 'accountID': 2382277312029984,
 'ability2Level': 1,
 'health': 1254,
 'ultimateName': 'YuumiR',
 'summonerSpell2Name': 'SummonerExhaust',
 'cooldownReduction': 0,
 'goldStats': {'assist': 142,
  'killPalisade': 262,
  'killMinion': 21,
  'killStructure': 416,
  'misc': 170,
  'itemSold': 20,
  'killChampion': 600,
  'supportItem': 680,
  'ambient': 2438,
  'starting': 500},
 'ma

In [152]:
event['teams'][0]

{'inhibKills': 0,
 'towerKills': 0,
 'teamID': 100,
 'baronKills': 0,
 'dragonKills': 0,
 'assists': 3,
 'championsKills': 1,
 'totalGold': 31061,
 'deaths': 5}

In [232]:
game_data[-3]

{'eventTime': '2023-06-22T23:09:39.168Z',
 'eventType': 'stats_update',
 'platformGameId': 'ESPORTSTMNT02:3207804',
 'gameTime': 1606009,
 'stageID': 1,
 'participants': [{'ability4CooldownRemaining': 35.4078369140625,
   'magicPenetrationPercent': 0,
   'alive': False,
   'primaryAbilityResource': 13,
   'spellVamp': 0,
   'participantID': 1,
   'lifeSteal': 0,
   'primaryAbilityResourceRegen': 30,
   'ability1CooldownRemaining': 0,
   'XPForNextLevel': 13020,
   'summonerSpell2CooldownRemaining': 0,
   'healthMax': 3891,
   'magicResist': 91,
   'ability3CooldownRemaining': 0,
   'ability3Level': 2,
   'ability2CooldownRemaining': 0,
   'summonerSpell1Name': 'S12_SummonerTeleportUpgrade',
   'teamID': 100,
   'currentGold': 880,
   'healthRegen': 34,
   'ability4Level': 2,
   'level': 14,
   'ability1Level': 5,
   'armorPenetration': 0,
   'accountID': 2382277692401216,
   'ability2Level': 5,
   'health': 0,
   'ultimateName': 'SionR',
   'summonerSpell2Name': 'SummonerFlash',
   'co

In [184]:
event = game_data[2001]

participant_data = event['participants'][0]


event_time = event['eventTime']
def extract_participant_stats(participant_data):
    """
    Extracts basic stats from participant data
    """
    participant_id = str(participant_data['participantID'])
    team_id = str(participant_data['teamID'])
    participant_stats_basic = {'XP': participant_data['XP'], 'totalGold': participant_data['totalGold'], 'currentGold':participant_data['currentGold']}
    participant_stats_frames = participant_data['stats']
    participant_stats_granular = extract_participants_granular_stats(participant_stats_frames)
    
    # Combine the basic and granular stats
    participant_stats = {'participant_id': participant_id, 'team_id': team_id, **participant_stats_basic, **participant_stats_granular}
    return participant_stats
    
def extract_participants_granular_stats(stats_frames):
    """
    Extracts granular stats from participant data (e.g. minions killed, jungle minions killed, KDA, etc.)
    :param stats_frame: the 'stats' item from the 'status_update' event 
    :return: a dictionary of granular stats
    """
    stat_record = {}
    stats_to_record = {'MINIONS_KILLED', 'NEUTRAL_MINIONS_KILLED', 'NEUTRAL_MINIONS_KILLED_YOUR_JUNGLE', 'NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE', 'CHAMPIONS_KILLED','NUM_DEATHS','ASSISTS',
                       'WARD_PLACED','WARD_KILLED','VISION_SCORE','TOTAL_DAMAGE_DEALT_TO_CHAMPIONS', 'TOTAL_DAMAGE_TAKEN', 'TOTAL_DAMAGE_DEALT_TO_BUILDINGS', 'TOTAL_DAMAGE_DEALT_TO_OBJECTIVES'}
    # other stats to consider 
    # ['TOTAL_TIME_CROWD_CONTROL_DEALT_TO_CHAMPIONS', 'TOTAL_HEAL_ON_TEAMMATES', 'TOTAL_DAMAGE_SHIELDED_ON_TEAMMATES']
    for stat in stats_frames:
        stat_name = stat['name']
        if stat_name in stats_to_record:
            stat_record[stat_name] = stat['value']
    return stat_record

# Now need to convert frame_stats into a single row of data for each team (100 and 200)
def create_participant_features(frame_stats):
    """
    Creates higher-order features for each participant (e.g. KDA, minions killed, etc.) 
    :param frame_stats: a dataframe with stats for each participant for a single team at a single point in time
    :return: a dataframe with features for each participant
    """
    # Create features for each participant
    # For KDA, we need to add 0.1 to the denominator to avoid dividing by 0
    num_minutes = frame_stats['event_time'] / 60
    frame_stats['KDA'] = (frame_stats['CHAMPIONS_KILLED'] + frame_stats['ASSISTS']) / (frame_stats['NUM_DEATHS'] + 1.0)
    frame_stats['cs_per_min'] = (frame_stats['MINIONS_KILLED'] + frame_stats['NEUTRAL_MINIONS_KILLED']) / num_minutes
    frame_stats['xp_per_min'] = frame_stats['XP'] / num_minutes
    frame_stats['gold_per_min'] = frame_stats['totalGold'] / num_minutes
    frame_stats['vision_per_min'] = frame_stats['VISION_SCORE'] / num_minutes
    frame_stats['wards_per_min'] = (frame_stats['WARD_PLACED'] + frame_stats['WARD_KILLED']) / num_minutes
    frame_stats['trade_efficiency'] = frame_stats['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS'] / (frame_stats['TOTAL_DAMAGE_TAKEN'] + 1.0)  # Higher for poke-heavy champions
    frame_stats['damage_to_champions_per_min'] = frame_stats['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS'] / num_minutes
    frame_stats['damage_to_buildings_per_min'] = frame_stats['TOTAL_DAMAGE_DEALT_TO_BUILDINGS'] / num_minutes
    frame_stats['gold_spent'] = frame_stats['totalGold'] - frame_stats['currentGold']
    frame_stats['share_of_minions_stolen'] = frame_stats['NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE'] / (frame_stats['NEUTRAL_MINIONS_KILLED'] + 1.0)
    frame_stats['share_of_team_gold'] = frame_stats['totalGold'] / frame_stats['totalGold'].sum()
    frame_stats['share_of_team_xp'] = frame_stats['XP'] / frame_stats['XP'].sum()
    frame_stats['share_of_team_damage_to_champions'] = frame_stats['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS'] / frame_stats['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS'].sum()
    frame_stats['share_of_team_damage_to_buildings'] = frame_stats['TOTAL_DAMAGE_DEALT_TO_BUILDINGS'] / frame_stats['TOTAL_DAMAGE_DEALT_TO_BUILDINGS'].sum()
    frame_stats['share_of_team_damage_to_objectives'] = frame_stats['TOTAL_DAMAGE_DEALT_TO_OBJECTIVES'] / frame_stats['TOTAL_DAMAGE_DEALT_TO_OBJECTIVES'].sum()
    return frame_stats

team_features_from_event_data = ['inhibKills', 'towerKills', 'baronKills', 'dragonKills', 'assists', 'championsKills', 'totalGold', 'deaths']
team_features_from_frame_stats = ['MINIONS_KILLED', 'NEUTRAL_MINIONS_KILLED', 'VISION_SCORE']

def create_team_features(frame_stats, event_data):
    """
    Creates team-level features (e.g. total gold, total share of dragons/barons taken, etc.)
    :param frame_stats: A previously generated dataframe with features for each participant 
    :param event_data: The event data for the current event 
    :return: 
    """
    # Extract the team_features for each team based on event_data['teams'][i][feature]
    # Then compute the share of each feature for each team (whenever both features are zero, set the share to 0.5)
    # Then add the share of each feature to the frame_stats dataframe
    
    # First extract the team features from event_data
    team_features = {'100': {}, '200': {}}
    for team in event_data['teams']:
        team_id = str(team['teamID'])
        for feature in team_features_from_event_data:
            team_features[team_id][feature] = team[feature]
    
    # Compute the data for each team as the sum of the data for each participant on each team 
    for team in team_features:
        for feature in team_features_from_frame_stats:
            team_features[team][feature] = frame_stats[frame_stats['team_id'] == team][feature].sum()
    
    # Now compute the share of each feature for each team, setting the share to 0.5 if they are both 0
    for team in team_features:
        for feature in team_features_from_event_data + team_features_from_frame_stats:
            total_value = (team_features['100'][feature] + team_features['200'][feature])
            if total_value == 0:
                team_features[team]['team_share_of_' + feature] = 0.5
            else:
                team_features[team]['team_share_of_' + feature] = team_features[team][feature] / total_value
                
    # Drop all features that are not 'share_of_' features
    for team in team_features:
        for feature in team_features_from_event_data + team_features_from_frame_stats:
            del team_features[team][feature]
    return team_features
            

def create_diff_features(frame_stats):
    # Create features based on the difference between the player on the row and the player on the other team (row +5)
    # Do this for the first 5 players
    # For diff metrics, the value for the other team is the negative of the values for the first team 
    # For share metrics, the value for the other team is 1- the value for the first team
    
    # DIFF METRICS
    # Overall relative performance indicators 
    frame_stats['KDA_diff'] = (frame_stats['KDA'] - frame_stats['KDA'].shift(-5))  
    frame_stats['cs_per_min_diff'] = (frame_stats['cs_per_min'] - frame_stats['cs_per_min'].shift(-5))
    frame_stats['xp_per_min_diff'] = (frame_stats['xp_per_min'] - frame_stats['xp_per_min'].shift(-5))
    frame_stats['gold_per_min_diff'] = (frame_stats['gold_per_min'] - frame_stats['gold_per_min'].shift(-5))
    frame_stats['vision_per_min_diff'] = (frame_stats['vision_per_min'] - frame_stats['vision_per_min'].shift(-5))
    frame_stats['wards_per_min_diff'] = (frame_stats['wards_per_min'] - frame_stats['wards_per_min'].shift(-5))
    
    # Lane trade indicators
    frame_stats['trade_efficiency_diff'] = (frame_stats['trade_efficiency'] - frame_stats['trade_efficiency'].shift(-5))
    frame_stats['damage_to_champions_per_min_diff'] = (frame_stats['damage_to_champions_per_min'] - frame_stats['damage_to_champions_per_min'].shift(-5))
    frame_stats['damage_to_buildings_per_min_diff'] = (frame_stats['damage_to_buildings_per_min'] - frame_stats['damage_to_buildings_per_min'].shift(-5))
    
    # SHARE METRICS
    # Opponent dominance indicators
    frame_stats['lane_cs_dominance'] = (frame_stats['cs_per_min'] / (frame_stats['cs_per_min'] + frame_stats['cs_per_min'].shift(-5)))
    frame_stats['lane_damage_to_champions_dominance'] = (frame_stats['damage_to_champions_per_min'] / 
                                                         (1.0 + frame_stats['damage_to_champions_per_min'] + frame_stats['damage_to_champions_per_min'].shift(-5)))
    frame_stats['lane_damage_to_buildings_dominance'] = (frame_stats['damage_to_buildings_per_min'] / 
                                                         (1.0 + frame_stats['damage_to_buildings_per_min'] + frame_stats['damage_to_buildings_per_min'].shift(-5)))
    
    # AGGREGATE METRICS
    # Now get the values for the other team (row +5) by either doing negative or 1- the values we just computed. But ONLY operate on rows 5-9
    # DIFF METRICS
    frame_stats.loc[5:10, 'KDA_diff'] = -frame_stats.loc[0:4, 'KDA_diff'].values
    frame_stats.loc[5:10, 'cs_per_min_diff'] = -frame_stats.loc[0:4, 'cs_per_min_diff'].values
    frame_stats.loc[5:10, 'xp_per_min_diff'] = -frame_stats.loc[0:4, 'xp_per_min_diff'].values
    frame_stats.loc[5:10, 'gold_per_min_diff'] = -frame_stats.loc[0:4, 'gold_per_min_diff'].values
    frame_stats.loc[5:10, 'vision_per_min_diff'] = -frame_stats.loc[0:4, 'vision_per_min_diff'].values
    frame_stats.loc[5:10, 'trade_efficiency_diff'] = -frame_stats.loc[0:4, 'trade_efficiency_diff'].values
    frame_stats.loc[5:10, 'damage_to_champions_per_min_diff'] = -frame_stats.loc[0:4, 'damage_to_champions_per_min_diff'].values
    frame_stats.loc[5:10, 'damage_to_buildings_per_min_diff'] = -frame_stats.loc[0:4, 'damage_to_buildings_per_min_diff'].values
    # SHARE METRICS
    frame_stats.loc[5:10, 'lane_cs_dominance'] = 1 - frame_stats.loc[0:4, 'lane_cs_dominance'].values
    frame_stats.loc[5:10, 'lane_damage_to_champions_dominance'] = 1 - frame_stats.loc[0:4, 'lane_damage_to_champions_dominance'].values
    frame_stats.loc[5:10, 'lane_damage_to_buildings_dominance'] = 1 - frame_stats.loc[0:4, 'lane_damage_to_buildings_dominance'].values
    
    return frame_stats


participant_mapping = {
    '1': 'top',
    '2': 'jungle',
    '3': 'mid',
    '4': 'bot',
    '5': 'support',
    '6': 'top',
    '7': 'jungle',
    '8': 'mid',
    '9': 'bot',
    '10': 'support'
}
def melt_frame_stats(frame_stats):
    """
    Melts the frame_stats dataframe so that each row is a single feature for all participants
    """
    # Rename the participant_id such that it maps to ["top","jungle","mid","bot","support"] for each team
    frame_stats['participant_id'] = frame_stats['participant_id'].map(participant_mapping)
    frame_stats = frame_stats.melt(id_vars = ['participant_id'], value_vars = frame_stats.columns)
    frame_stats['name'] = frame_stats['participant_id'] + '_' + frame_stats['variable']
    frame_stats = frame_stats[['name','value']].set_index('name').transpose()
    return frame_stats


In [165]:
event

{'eventTime': '2023-06-22T23:02:24.156Z',
 'eventType': 'stats_update',
 'platformGameId': 'ESPORTSTMNT02:3207804',
 'gameTime': 1171003,
 'stageID': 1,
 'participants': [{'ability4CooldownRemaining': 0,
   'magicPenetrationPercent': 0,
   'alive': True,
   'primaryAbilityResource': 699,
   'spellVamp': 0,
   'participantID': 1,
   'lifeSteal': 0,
   'primaryAbilityResourceRegen': 27,
   'ability1CooldownRemaining': 2.568115234375,
   'XPForNextLevel': 9960,
   'summonerSpell2CooldownRemaining': 0,
   'healthMax': 3191,
   'magicResist': 77,
   'ability3CooldownRemaining': 0.8946533203125,
   'ability3Level': 1,
   'ability2CooldownRemaining': 10.202392578125,
   'summonerSpell1Name': 'S12_SummonerTeleportUpgrade',
   'teamID': 100,
   'currentGold': 780,
   'healthRegen': 38,
   'ability4Level': 2,
   'level': 12,
   'ability1Level': 5,
   'armorPenetration': 0,
   'accountID': 2382277692401216,
   'ability2Level': 4,
   'health': 2260,
   'ultimateName': 'SionR',
   'summonerSpell2Na

In [166]:
participant_data

{'ability4CooldownRemaining': 0,
 'magicPenetrationPercent': 0,
 'alive': True,
 'primaryAbilityResource': 699,
 'spellVamp': 0,
 'participantID': 1,
 'lifeSteal': 0,
 'primaryAbilityResourceRegen': 27,
 'ability1CooldownRemaining': 2.568115234375,
 'XPForNextLevel': 9960,
 'summonerSpell2CooldownRemaining': 0,
 'healthMax': 3191,
 'magicResist': 77,
 'ability3CooldownRemaining': 0.8946533203125,
 'ability3Level': 1,
 'ability2CooldownRemaining': 10.202392578125,
 'summonerSpell1Name': 'S12_SummonerTeleportUpgrade',
 'teamID': 100,
 'currentGold': 780,
 'healthRegen': 38,
 'ability4Level': 2,
 'level': 12,
 'ability1Level': 5,
 'armorPenetration': 0,
 'accountID': 2382277692401216,
 'ability2Level': 4,
 'health': 2260,
 'ultimateName': 'SionR',
 'summonerSpell2Name': 'SummonerFlash',
 'cooldownReduction': 0,
 'goldStats': {'assist': 20,
  'killPalisade': 350,
  'misc': 45,
  'itemSold': 5,
  'killMinion': 3201,
  'starting': 500,
  'ambient': 2164},
 'magicPenetrationPercentBonus': 0,


In [192]:
individual_stats = []
for participant_data in event['participants']:
    individual_stats.append(extract_participant_stats(participant_data))

# Convert the list of dictionaries into a single dataframe and ensure that it's sorted by partipant_id when evaluated as a number
frame_stats = pd.DataFrame.from_dict(individual_stats).sort_values(by=['participant_id'], key=lambda x: x.astype(int))

# Add the time column
event_time = 600
frame_stats['event_time'] = event_time

frame_stats_100 = frame_stats[frame_stats['team_id'] == '100']
frame_stats_100 = create_participant_features(frame_stats[frame_stats['team_id'] == '100'])
frame_stats_200 = frame_stats[frame_stats['team_id'] == '200']
frame_stats_200 = create_participant_features(frame_stats[frame_stats['team_id'] == '200'])

frame_stats = pd.concat([frame_stats_100, frame_stats_200])
# Get the team features
team_features_stats = create_team_features(frame_stats, event)
# Get the lane difference features 
frame_stats = create_diff_features(frame_stats)

# Lastly turn these into per-player features by splitting again by team_id, pivoting on participant_id, and then joining back together
frame_stats_100 = frame_stats[frame_stats['team_id'] == '100'].drop('team_id',axis=1)
frame_stats_200 = frame_stats[frame_stats['team_id'] == '200'].drop('team_id',axis=1)

# Pivot and convert to dictionary format 
frame_stats_100 = melt_frame_stats(frame_stats_100)
frame_stats_100 = {x:y for x, y in zip(frame_stats_100.columns.values, frame_stats_100.values[0])}
all_stats_100 = {**team_features_stats['100'], **frame_stats_100}
all_stats_100 = {x + f'_at_{event_time/60.0}': y for x, y in all_stats_100.items()}

frame_stats_200 = melt_frame_stats(frame_stats_200)
frame_stats_200 = {x:y for x, y in zip(frame_stats_200.columns.values, frame_stats_200.values[0])}
all_stats_200 = {**team_features_stats['200'], **frame_stats_200}
all_stats_200 = {x + f'_at_{event_time/60.0}': y for x, y in all_stats_200.items()}

C:\Users\brian\AppData\Local\Temp\ipykernel_2768\2373307094.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_stats['KDA'] = (frame_stats['CHAMPIONS_KILLED'] + frame_stats['ASSISTS']) / (frame_stats['NUM_DEATHS'] + 1.0)
C:\Users\brian\AppData\Local\Temp\ipykernel_2768\2373307094.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_stats['cs_per_min'] = (frame_stats['MINIONS_KILLED'] + frame_stats['NEUTRAL_MINIONS_KILLED']) / num_minutes
C:\Users\brian\AppData\Local\Temp\ipykernel_2768\2373307

In [194]:
all_stats_100

{'team_share_of_inhibKills_at_10.0': 0.5,
 'team_share_of_towerKills_at_10.0': 0.0,
 'team_share_of_baronKills_at_10.0': 0.5,
 'team_share_of_dragonKills_at_10.0': 0.0,
 'team_share_of_assists_at_10.0': 0.2,
 'team_share_of_championsKills_at_10.0': 0.16666666666666666,
 'team_share_of_totalGold_at_10.0': 0.46154417664715147,
 'team_share_of_deaths_at_10.0': 0.8333333333333334,
 'team_share_of_MINIONS_KILLED_at_10.0': 0.4876805437553101,
 'team_share_of_NEUTRAL_MINIONS_KILLED_at_10.0': 0.4531249948777269,
 'team_share_of_VISION_SCORE_at_10.0': 0.453499424082263,
 'top_XP_at_10.0': 9266.0,
 'jungle_XP_at_10.0': 7207.0,
 'mid_XP_at_10.0': 9708.0,
 'bot_XP_at_10.0': 7765.0,
 'support_XP_at_10.0': 5107.0,
 'top_totalGold_at_10.0': 6280.0,
 'jungle_totalGold_at_10.0': 6138.0,
 'mid_totalGold_at_10.0': 6845.0,
 'bot_totalGold_at_10.0': 7648.0,
 'support_totalGold_at_10.0': 4150.0,
 'top_currentGold_at_10.0': 780.0,
 'jungle_currentGold_at_10.0': 948.0,
 'mid_currentGold_at_10.0': 710.0,
 'bot

In [187]:
team_features_stats

{'team_100': {'team_share_of_inhibKills': 0.5,
  'team_share_of_towerKills': 0.0,
  'team_share_of_baronKills': 0.5,
  'team_share_of_dragonKills': 0.0,
  'team_share_of_assists': 0.2,
  'team_share_of_championsKills': 0.16666666666666666,
  'team_share_of_totalGold': 0.46154417664715147,
  'team_share_of_deaths': 0.8333333333333334,
  'team_share_of_MINIONS_KILLED': 0.4876805437553101,
  'team_share_of_NEUTRAL_MINIONS_KILLED': 0.4531249948777269,
  'team_share_of_VISION_SCORE': 0.453499424082263},
 'team_200': {'team_share_of_inhibKills': 0.5,
  'team_share_of_towerKills': 1.0,
  'team_share_of_baronKills': 0.5,
  'team_share_of_dragonKills': 1.0,
  'team_share_of_assists': 0.8,
  'team_share_of_championsKills': 0.8333333333333334,
  'team_share_of_totalGold': 0.5384558233528485,
  'team_share_of_deaths': 0.16666666666666666,
  'team_share_of_MINIONS_KILLED': 0.5123194562446899,
  'team_share_of_NEUTRAL_MINIONS_KILLED': 0.546875005122273,
  'team_share_of_VISION_SCORE': 0.546500575917

In [8]:
from datetime import datetime
import pandas as pd
import numpy as np

class FrameFeaturesAtTimeGenerator:
    def __init__(self, event, event_time):
        self.event = event
        self.event_time = event_time  # Must be time in SECONDS, not minutes 
        
        self.team_features_from_event_data = ['inhibKills', 'towerKills', 'baronKills', 'dragonKills', 'assists', 'championsKills', 'totalGold', 'deaths']
        self.team_features_from_frame_stats = ['MINIONS_KILLED', 'NEUTRAL_MINIONS_KILLED', 'VISION_SCORE']
        self.participant_mapping = {
            '1': 'top',
            '2': 'jungle',
            '3': 'mid',
            '4': 'bot',
            '5': 'support',
            '6': 'top',
            '7': 'jungle',
            '8': 'mid',
            '9': 'bot',
            '10': 'support'
        }
        
    def extract_participant_stats(self, participant_data):
        """
        Extracts basic stats from participant data
        """
        participant_id = str(participant_data['participantID'])
        team_id = str(participant_data['teamID'])
        participant_stats_basic = {'XP': participant_data['XP'], 'totalGold': participant_data['totalGold'], 'currentGold':participant_data['currentGold']}
        participant_stats_frames = participant_data['stats']
        participant_stats_granular = self.extract_participants_granular_stats(participant_stats_frames)
        
        # Combine the basic and granular stats
        participant_stats = {'participant_id': participant_id, 'team_id': team_id, **participant_stats_basic, **participant_stats_granular}
        return participant_stats
        
    def extract_participants_granular_stats(self, stats_frames):
        """
        Extracts granular stats from participant data (e.g. minions killed, jungle minions killed, KDA, etc.)
        :param stats_frame: the 'stats' item from the 'status_update' event 
        :return: a dictionary of granular stats
        """
        stat_record = {}
        stats_to_record = {'MINIONS_KILLED', 'NEUTRAL_MINIONS_KILLED', 'NEUTRAL_MINIONS_KILLED_YOUR_JUNGLE', 'NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE', 'CHAMPIONS_KILLED','NUM_DEATHS','ASSISTS',
                           'WARD_PLACED','WARD_KILLED','VISION_SCORE','TOTAL_DAMAGE_DEALT_TO_CHAMPIONS', 'TOTAL_DAMAGE_TAKEN', 'TOTAL_DAMAGE_DEALT_TO_BUILDINGS', 'TOTAL_DAMAGE_DEALT_TO_OBJECTIVES'}
        # other stats to consider 
        # ['TOTAL_TIME_CROWD_CONTROL_DEALT_TO_CHAMPIONS', 'TOTAL_HEAL_ON_TEAMMATES', 'TOTAL_DAMAGE_SHIELDED_ON_TEAMMATES']
        for stat in stats_frames:
            stat_name = stat['name']
            if stat_name in stats_to_record:
                stat_record[stat_name] = stat['value']
        return stat_record
    
    # Now need to convert frame_stats into a single row of data for each team (100 and 200)
    def create_participant_features(self, frame_stats):
        """
        Creates higher-order features for each participant (e.g. KDA, minions killed, etc.) 
        :param frame_stats: a dataframe with stats for each participant for a single team at a single point in time
        :return: a dataframe with features for each participant
        """
        # Create features for each participant
        # For KDA, we need to add 0.1 to the denominator to avoid dividing by 0
        num_minutes = frame_stats['event_time'] / 60
        frame_stats['KDA'] = (frame_stats['CHAMPIONS_KILLED'] + frame_stats['ASSISTS']) / (frame_stats['NUM_DEATHS'] + 1.0)
        frame_stats['cs_per_min'] = (frame_stats['MINIONS_KILLED'] + frame_stats['NEUTRAL_MINIONS_KILLED']) / num_minutes
        frame_stats['xp_per_min'] = frame_stats['XP'] / num_minutes
        frame_stats['gold_per_min'] = frame_stats['totalGold'] / num_minutes
        frame_stats['vision_per_min'] = frame_stats['VISION_SCORE'] / num_minutes
        frame_stats['wards_per_min'] = (frame_stats['WARD_PLACED'] + frame_stats['WARD_KILLED']) / num_minutes
        frame_stats['trade_efficiency'] = frame_stats['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS'] / (frame_stats['TOTAL_DAMAGE_TAKEN'] + 1.0)  # Higher for poke-heavy champions
        frame_stats['damage_to_champions_per_min'] = frame_stats['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS'] / num_minutes
        frame_stats['damage_to_buildings_per_min'] = frame_stats['TOTAL_DAMAGE_DEALT_TO_BUILDINGS'] / num_minutes
        frame_stats['gold_spent'] = frame_stats['totalGold'] - frame_stats['currentGold']
        frame_stats['share_of_minions_stolen'] = frame_stats['NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE'] / (frame_stats['NEUTRAL_MINIONS_KILLED'] + 1.0)
        frame_stats['share_of_team_gold'] = frame_stats['totalGold'] / frame_stats['totalGold'].sum()
        frame_stats['share_of_team_xp'] = frame_stats['XP'] / frame_stats['XP'].sum()
        frame_stats['share_of_team_damage_to_champions'] = frame_stats['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS'] / frame_stats['TOTAL_DAMAGE_DEALT_TO_CHAMPIONS'].sum()
        frame_stats['share_of_team_damage_to_buildings'] = frame_stats['TOTAL_DAMAGE_DEALT_TO_BUILDINGS'] / frame_stats['TOTAL_DAMAGE_DEALT_TO_BUILDINGS'].sum()
        frame_stats['share_of_team_damage_to_objectives'] = frame_stats['TOTAL_DAMAGE_DEALT_TO_OBJECTIVES'] / frame_stats['TOTAL_DAMAGE_DEALT_TO_OBJECTIVES'].sum()
        return frame_stats
    
    
    def create_team_features(self, frame_stats, event_data):
        """
        Creates team-level features (e.g. total gold, total share of dragons/barons taken, etc.)
        :param frame_stats: A previously generated dataframe with features for each participant 
        :param event_data: The event data for the current event 
        :return: 
        """
        # Extract the team_features for each team based on event_data['teams'][i][feature]
        # Then compute the share of each feature for each team (whenever both features are zero, set the share to 0.5)
        # Then add the share of each feature to the frame_stats dataframe
        
        # First extract the team features from event_data
        team_features = {'100': {}, '200': {}}
        for team in event_data['teams']:
            team_id = str(team['teamID'])
            for feature in self.team_features_from_event_data:
                team_features[team_id][feature] = team[feature]
        
        # Compute the data for each team as the sum of the data for each participant on each team 
        for team in team_features:
            for feature in self.team_features_from_frame_stats:
                team_features[team][feature] = frame_stats[frame_stats['team_id'] == team][feature].sum()
        
        # Now compute the share of each feature for each team, setting the share to 0.5 if they are both 0
        for team in team_features:
            for feature in self.team_features_from_event_data + self.team_features_from_frame_stats:
                total_value = (team_features['100'][feature] + team_features['200'][feature])
                if total_value == 0:
                    team_features[team]['team_share_of_' + feature] = 0.5
                else:
                    team_features[team]['team_share_of_' + feature] = team_features[team][feature] / total_value
                    
        # Drop all features that are not 'share_of_' features
        for team in team_features:
            for feature in self.team_features_from_event_data + self.team_features_from_frame_stats:
                del team_features[team][feature]
        return team_features
                
    
    def create_diff_features(self, frame_stats):
        # Create features based on the difference between the player on the row and the player on the other team (row +5)
        # Do this for the first 5 players
        # For diff metrics, the value for the other team is the negative of the values for the first team 
        # For share metrics, the value for the other team is 1- the value for the first team
        
        # DIFF METRICS
        # Overall relative performance indicators 
        frame_stats['KDA_diff'] = (frame_stats['KDA'] - frame_stats['KDA'].shift(-5))  
        frame_stats['cs_per_min_diff'] = (frame_stats['cs_per_min'] - frame_stats['cs_per_min'].shift(-5))
        frame_stats['xp_per_min_diff'] = (frame_stats['xp_per_min'] - frame_stats['xp_per_min'].shift(-5))
        frame_stats['gold_per_min_diff'] = (frame_stats['gold_per_min'] - frame_stats['gold_per_min'].shift(-5))
        frame_stats['vision_per_min_diff'] = (frame_stats['vision_per_min'] - frame_stats['vision_per_min'].shift(-5))
        frame_stats['wards_per_min_diff'] = (frame_stats['wards_per_min'] - frame_stats['wards_per_min'].shift(-5))
        
        # Lane trade indicators
        frame_stats['trade_efficiency_diff'] = (frame_stats['trade_efficiency'] - frame_stats['trade_efficiency'].shift(-5))
        frame_stats['damage_to_champions_per_min_diff'] = (frame_stats['damage_to_champions_per_min'] - frame_stats['damage_to_champions_per_min'].shift(-5))
        frame_stats['damage_to_buildings_per_min_diff'] = (frame_stats['damage_to_buildings_per_min'] - frame_stats['damage_to_buildings_per_min'].shift(-5))
        
        # SHARE METRICS
        # Opponent dominance indicators
        frame_stats['lane_cs_dominance'] = (frame_stats['cs_per_min'] / (frame_stats['cs_per_min'] + frame_stats['cs_per_min'].shift(-5)))
        frame_stats['lane_damage_to_champions_dominance'] = (frame_stats['damage_to_champions_per_min'] / 
                                                             (1.0 + frame_stats['damage_to_champions_per_min'] + frame_stats['damage_to_champions_per_min'].shift(-5)))
        frame_stats['lane_damage_to_buildings_dominance'] = (frame_stats['damage_to_buildings_per_min'] / 
                                                             (1.0 + frame_stats['damage_to_buildings_per_min'] + frame_stats['damage_to_buildings_per_min'].shift(-5)))
        
        # AGGREGATE METRICS
        # Now get the values for the other team (row +5) by either doing negative or 1- the values we just computed. But ONLY operate on rows 5-9
        # DIFF METRICS
        frame_stats.loc[5:10, 'KDA_diff'] = -frame_stats.loc[0:4, 'KDA_diff'].values
        frame_stats.loc[5:10, 'cs_per_min_diff'] = -frame_stats.loc[0:4, 'cs_per_min_diff'].values
        frame_stats.loc[5:10, 'xp_per_min_diff'] = -frame_stats.loc[0:4, 'xp_per_min_diff'].values
        frame_stats.loc[5:10, 'gold_per_min_diff'] = -frame_stats.loc[0:4, 'gold_per_min_diff'].values
        frame_stats.loc[5:10, 'vision_per_min_diff'] = -frame_stats.loc[0:4, 'vision_per_min_diff'].values
        frame_stats.loc[5:10, 'trade_efficiency_diff'] = -frame_stats.loc[0:4, 'trade_efficiency_diff'].values
        frame_stats.loc[5:10, 'damage_to_champions_per_min_diff'] = -frame_stats.loc[0:4, 'damage_to_champions_per_min_diff'].values
        frame_stats.loc[5:10, 'damage_to_buildings_per_min_diff'] = -frame_stats.loc[0:4, 'damage_to_buildings_per_min_diff'].values
        frame_stats.loc[5:10, 'wards_per_min_diff'] = -frame_stats.loc[0:4, 'wards_per_min_diff'].values
        # SHARE METRICS
        frame_stats.loc[5:10, 'lane_cs_dominance'] = 1 - frame_stats.loc[0:4, 'lane_cs_dominance'].values
        frame_stats.loc[5:10, 'lane_damage_to_champions_dominance'] = 1 - frame_stats.loc[0:4, 'lane_damage_to_champions_dominance'].values
        frame_stats.loc[5:10, 'lane_damage_to_buildings_dominance'] = 1 - frame_stats.loc[0:4, 'lane_damage_to_buildings_dominance'].values
        
        return frame_stats
    
    
    def melt_frame_stats(self, frame_stats):
        """
        Melts the frame_stats dataframe so that each row is a single feature for all participants
        """
        # Rename the participant_id such that it maps to ["top","jungle","mid","bot","support"] for each team
        frame_stats['participant_id'] = frame_stats['participant_id'].map(self.participant_mapping)
        frame_stats = frame_stats.melt(id_vars = ['participant_id'], value_vars = frame_stats.columns)
        frame_stats['name'] = frame_stats['participant_id'] + '_' + frame_stats['variable']
        frame_stats = frame_stats[['name','value']].set_index('name').transpose()
        return frame_stats
    
    def process_frame(self, time = None):
        individual_stats = []
        for participant_data in self.event['participants']:
            individual_stats.append(self.extract_participant_stats(participant_data))
        
        # Convert the list of dictionaries into a single dataframe and ensure that it's sorted by partipant_id when evaluated as a number
        frame_stats = pd.DataFrame.from_dict(individual_stats).sort_values(by=['participant_id'], key=lambda x: x.astype(int))
        
        # Add the time column
        frame_stats['event_time'] = self.event_time
        
        frame_stats_100 = frame_stats[frame_stats['team_id'] == '100']
        frame_stats_100 = self.create_participant_features(frame_stats[frame_stats['team_id'] == '100'])
        frame_stats_200 = frame_stats[frame_stats['team_id'] == '200']
        frame_stats_200 = self.create_participant_features(frame_stats[frame_stats['team_id'] == '200'])
        
        frame_stats = pd.concat([frame_stats_100, frame_stats_200])
        # Get the team features
        team_features_stats = self.create_team_features(frame_stats, event)
        # Get the lane difference features 
        frame_stats = self.create_diff_features(frame_stats)
        
        # Lastly turn these into per-player features by splitting again by team_id, pivoting on participant_id, and then joining back together
        frame_stats_100 = frame_stats[frame_stats['team_id'] == '100'].drop(['team_id', 'event_time'],axis=1)
        frame_stats_200 = frame_stats[frame_stats['team_id'] == '200'].drop(['team_id', 'event_time'],axis=1)
        
        # Pivot and convert to dictionary format 
        frame_stats_100 = self.melt_frame_stats(frame_stats_100)
        frame_stats_100 = {x:y for x, y in zip(frame_stats_100.columns.values, frame_stats_100.values[0])}
        all_stats_100 = {**team_features_stats['100'], **frame_stats_100}
        if time is None:
            time = round(self.event_time/60)
        all_stats_100 = {x + f'_at_{time}': y for x, y in all_stats_100.items()}
        
        frame_stats_200 = self.melt_frame_stats(frame_stats_200)
        frame_stats_200 = {x:y for x, y in zip(frame_stats_200.columns.values, frame_stats_200.values[0])}
        all_stats_200 = {**team_features_stats['200'], **frame_stats_200}
        all_stats_200 = {x + f'_at_{time}': y for x, y in all_stats_200.items()}
        
        return all_stats_100, all_stats_200

In [9]:
event = game_data[-2]

frame_processor = FrameFeaturesAtTimeGenerator(event, 600)
frame_stats_100, frame_stats_200 = frame_processor.process_frame('game_end')

In [10]:
len(frame_stats_200)

236

In [11]:
frame_stats_100

{'team_share_of_inhibKills_at_game_end': 1.0,
 'team_share_of_towerKills_at_game_end': 0.9090909090909091,
 'team_share_of_baronKills_at_game_end': 1.0,
 'team_share_of_dragonKills_at_game_end': 1.0,
 'team_share_of_assists_at_game_end': 0.921875,
 'team_share_of_championsKills_at_game_end': 0.8928571428571429,
 'team_share_of_totalGold_at_game_end': 0.5884210647641221,
 'team_share_of_deaths_at_game_end': 0.10714285714285714,
 'team_share_of_MINIONS_KILLED_at_game_end': 0.49703138252756573,
 'team_share_of_NEUTRAL_MINIONS_KILLED_at_game_end': 0.5745701975713059,
 'team_share_of_VISION_SCORE_at_game_end': 0.6249044831147744,
 'top_XP_at_game_end': 12995.0,
 'jungle_XP_at_game_end': 12213.0,
 'mid_XP_at_game_end': 11271.0,
 'bot_XP_at_game_end': 10866.0,
 'support_XP_at_game_end': 7815.0,
 'top_totalGold_at_game_end': 9197.0,
 'jungle_totalGold_at_game_end': 13249.0,
 'mid_totalGold_at_game_end': 9412.0,
 'bot_totalGold_at_game_end': 11993.0,
 'support_totalGold_at_game_end': 7201.0,
 '

In [29]:
class GameFeaturesGenerator:
    TIME_FORMAT = '%Y-%m-%dT%H:%M:%S.%fZ'

    @staticmethod
    def get_event_time(start_time, event_time):
        return (datetime.strptime(event_time, GameFeaturesGenerator.TIME_FORMAT) - start_time).total_seconds()

    @staticmethod
    def flip_team_id(team_id):
        """
        Flips team_id from 100 to 200 and vice versa. Needed for when processing turret fall events because we want to update the feature for the team that lost the turret
        """
        if team_id == 100:
            return 200
        elif team_id == 200:
            return 100
        else:
            print("Error: team_id is not 100 or 200")

    def __init__(self, game_data, mapping_data):
        self.game_data = game_data
        self.game_start_time = datetime.strptime(game_data[0]['eventTime'], GameFeaturesGenerator.TIME_FORMAT)
        self.team_id_mapping = {"100": mapping_data['teamMapping']['100'], "200": mapping_data['teamMapping']['200']}
        self.esports_game_id = mapping_data['esportsGameId']  # Looks like '110310652412257228'
        self.platform_game_id = mapping_data['platformGameId']  # Looks lik {'esportsGameId': '110310652412257228'
        # Create flags for first events (e.g. first turret kill, first dragon kill, etc.). These can only happen once (for both teams)
        self.first_herald_flag = True
        self.first_dragon_flag = True
        self.first_baron_flag = True
        self.first_turret_flag = True
        self.first_inhibitor_flag = True
        self.first_kill_flag = True

        # Create features for each team
        # EPIC MONSTER EVENT FEATURES
        self.IMPORTANT_MONSTER_TYPES = ["riftHerald", "dragon", "baron"]
        epic_monster_kill_event_features = [
            ["first_" + monster_type + "_ind", "first_" + monster_type + "_time", "num_" + monster_type] for
            monster_type
            in self.IMPORTANT_MONSTER_TYPES]
        epic_monster_kill_event_features = [item for sublist in epic_monster_kill_event_features for item in
                                            sublist]  # Flatten list of lists

        # BUILDING DESTROYED EVENT FEATURES
        self.IMPORTANT_BUILDING_TYPES = ["turret", "inhibitor"]
        building_destroyed_event_features = [
            ["first_" + building_type + "_ind", "first_" + building_type + "_time", "num_" + building_type] for
            building_type
            in self.IMPORTANT_BUILDING_TYPES]
        building_destroyed_event_features = [item for sublist in building_destroyed_event_features for item in
                                             sublist]  # Flatten list of lists

        # CHAMPION KILL EVENT FEATURES
        champion_kill_event_features = ["first_kill_ind", "first_kill_time", "num_kills"]

        # GAME METADATA FEATURES
        # TODO: Also need a feature for whether this was a domestic or international game (e.g. LCS vs. Worlds)
        game_metadata_features = ["game_end_time"]

        all_features = (["platformGameId", "esportsGameId", "team_id", "start_time", "outcome"] +
                        epic_monster_kill_event_features + building_destroyed_event_features + champion_kill_event_features + game_metadata_features)
        self.team_features = {"100": {feature: np.nan for feature in all_features},
                              "200": {feature: np.nan for feature in all_features}}
        # Set all "num_" features to 0
        for team_id in ["100", "200"]:
            for feature in all_features:
                if feature.startswith("num_"):
                    self.team_features[team_id][feature] = 0

    def process_epic_monster_kill_event(self, event, team_features):
        monster_type = event['monsterType']
        if monster_type in self.IMPORTANT_MONSTER_TYPES:
            event_time = event['eventTime']
            time_of_kill = GameFeaturesGenerator.get_event_time(self.game_start_time, event_time)
            killer_team_id = str(event['killerTeamID'])
            victim_team_id = str(GameFeaturesGenerator.flip_team_id(event['killerTeamID']))
            if self.first_herald_flag:
                team_features[killer_team_id]['first_riftHerald_ind'] = 1
                team_features[victim_team_id]['first_riftHerald_ind'] = 0
                team_features[killer_team_id]['first_riftHerald_time'] = time_of_kill
                self.first_herald_flag = False
            if self.first_dragon_flag:
                team_features[killer_team_id]['first_dragon_ind'] = 1
                team_features[victim_team_id]['first_dragon_ind'] = 0
                team_features[killer_team_id]['first_dragon_time'] = time_of_kill
                self.first_dragon_flag = False
            if self.first_baron_flag:
                team_features[killer_team_id]['first_baron_ind'] = 1
                team_features[victim_team_id]['first_baron_ind'] = 0
                team_features[killer_team_id]['first_baron_time'] = time_of_kill
                self.first_baron_flag = False
            team_features[killer_team_id]['num_' + monster_type] += 1

    def process_building_destroyed_event(self, event, team_features):
        building_type = event['buildingType']
        if building_type in self.IMPORTANT_BUILDING_TYPES:
            event_time = event['eventTime']
            time_of_event = GameFeaturesGenerator.get_event_time(self.game_start_time, event_time)
            team_id = str(GameFeaturesGenerator.flip_team_id(event['teamID']))
            victim_team_id = str(event['teamID'])
            if self.first_turret_flag:
                team_features[team_id]['first_turret_ind'] = 1
                team_features[victim_team_id]['first_turret_ind'] = 0
                team_features[team_id]['first_turret_time'] = time_of_event
                self.first_turret_flag = False
            if self.first_inhibitor_flag:
                team_features[team_id]['first_inhibitor_ind'] = 1
                team_features[victim_team_id]['first_inhibitor_ind'] = 0
                team_features[team_id]['first_inhibitor_time'] = time_of_event
                self.first_inhibitor_flag = False
            team_features[team_id]['num_' + building_type] += 1

    def process_champion_kill_event(self, event, team_features):
        event_time = event['eventTime']
        time_of_event = GameFeaturesGenerator.get_event_time(self.game_start_time, event_time)
        killer_team_id = str(event['killerTeamID'])
        victim_team_id = str(GameFeaturesGenerator.flip_team_id(event['killerTeamID']))
        if self.first_kill_flag:
            team_features[killer_team_id]['first_kill_ind'] = 1
            team_features[victim_team_id]['first_kill_ind'] = 0
            team_features[killer_team_id]['first_kill_time'] = time_of_event
            self.first_kill_flag = False
        team_features[killer_team_id]['num_kills'] += 1

    def process_game_end_event(self, event, team_features):
        # We only call this method for the final event, so if for some reason there is no game_end event then discard the data
        if event['eventType'] == "game_end":
            event_time = event['eventTime']
            time_of_event = GameFeaturesGenerator.get_event_time(self.game_start_time, event_time)
            team_features['100']['game_end_time'] = time_of_event
            team_features['200']['game_end_time'] = time_of_event
            winning_team = event['winningTeam']
            losing_team = GameFeaturesGenerator.flip_team_id(winning_team)
            team_features[str(winning_team)]['outcome'] = 1
            team_features[str(losing_team)]['outcome'] = 0
        else:
            raise Exception("Error: event is not a game_end event")

    def process_game(self):
        """
        Loops through all events in the game and updates the team features as they occur
        :return: a pandas dataframe with the features for each team and label "outcome"
        """
        self.time_flags = [t for t in [7, 14, 20, 30]]  # 999 is a placeholder for the game_end flag
        frame_stats_100_dict = {str(time): None for time in self.time_flags+['game_end']}  # {'7': None, '14': None, '20': None, '30': None, 'game_end': None}
        frame_stats_200_dict = {str(time): None for time in self.time_flags+['game_end']}
        
        self.time_flags.append(999)
        for event in self.game_data:
            if event['eventType'] == "epic_monster_kill":
                self.process_epic_monster_kill_event(event, self.team_features)
            elif event['eventType'] == "building_destroyed":
                self.process_building_destroyed_event(event, self.team_features)
            elif event['eventType'] == "champion_kill":
                self.process_champion_kill_event(event, self.team_features)
            elif event['eventType'] == "game_end":
                self.process_game_end_event(event, self.team_features)
            elif event['eventType'] == "stats_update": 
                """
                Check the event time and process this frame if it is the first "stats_update" past each flag (7, 14, 20, 30)
                - 7: midlaners hit lv. 6 and may roam, dragons spawns at 5 rift herald spawn at 8 
                - 14: tower plates fall, unleashed TP activates
                - 20: Baron spawns 
                - 30: Very late game (laners likely full build with ~300 cs). Usually baron/elder is taken and game ends soon after.
                """ 
                event_time = GameFeaturesGenerator.get_event_time(self.game_start_time, event['eventTime'])
                if (event_time >= self.time_flags[0]*60) or event['gameOver']:
                    if event['gameOver']:
                        time_for_stat_label = 'game_end'
                        # print("Game ended at " + str(event_time))
                    else:
                        time_for_stat_label = str(self.time_flags.pop(0))
                    frame_processor = FrameFeaturesAtTimeGenerator(event, event_time)  # Requires the time for stat per min computations
                    frame_stats_100, frame_stats_200 = frame_processor.process_frame(time_for_stat_label)  # Requires time solely for labeling purposes
                    frame_stats_100_dict[time_for_stat_label] = frame_stats_100
                    frame_stats_200_dict[time_for_stat_label] = frame_stats_200
        
        # If any frames remain, then copy the last frame for each remaining time flag and fill the values with NA 
        self.time_flags.remove(999)
        for time_flag_remaining in self.time_flags:
            template_frame = deepcopy(frame_stats_100_dict['game_end'])
            # Replace the 'game_end' part of the name with the current time flag
            template_frame = {x.replace('game_end', str(time_flag_remaining)):y for x, y in template_frame.items()}
            # Replace the values with na to indicate that the game did not arrive at that state
            template_frame = {x:np.nan for x in template_frame}
            frame_stats_100_dict[str(time_flag_remaining)] = template_frame
            frame_stats_200_dict[str(time_flag_remaining)] = template_frame
        
        
        # Lastly, unpack all features into a single dictionary in order of the dictionary keys, ending with the game_end features
        frame_stats_100_full = {}
        frame_stats_200_full = {}
        for time_flag in frame_stats_100_dict:
            frame_stats_100_full = {**frame_stats_100_full, **frame_stats_100_dict[time_flag]}
            frame_stats_200_full = {**frame_stats_200_full, **frame_stats_200_dict[time_flag]}
        # Add the frame_stats to the team_features
        self.team_features['100'] = {**self.team_features['100'], **frame_stats_100_full}
        self.team_features['200'] = {**self.team_features['200'], **frame_stats_200_full}
        
        # Now we have all the datapoints, we can create a dataframe with the team ID, label, start_time, and features
        # First assign the rest of the metadata
        rows = []
        for team_id in ["100", "200"]:
            # Assign metadata (same for both teams)
            self.team_features[team_id]['platformGameId'] = self.platform_game_id
            self.team_features[team_id]['esportsGameId'] = self.esports_game_id
            self.team_features[team_id]['start_time'] = self.game_start_time

            # Assessing team-specific data
            self.team_features[team_id]['team_id'] = self.team_id_mapping[team_id]
            rows.append(pd.DataFrame.from_dict(self.team_features[team_id], orient='index').transpose())
        return pd.concat(rows, ignore_index=True)



In [76]:
# Read in teams data
with open("teams.json", "r") as json_file:
   teams_data = json.load(json_file)

In [77]:
teams_data

[{'team_id': '107582169874155554',
  'name': "God's Plan",
  'acronym': 'GDP',
  'slug': 'gods-plan'},
 {'team_id': '105550059790656435',
  'name': 'T1 Esports Academy',
  'acronym': 'T1',
  'slug': 't1-challengers'},
 {'team_id': '103535282143744679',
  'name': 'Dark Passage Akademi',
  'acronym': 'DP',
  'slug': 'dark-passage-akademi'},
 {'team_id': '109637393694097670',
  'name': 'Team Heretics',
  'acronym': 'TH',
  'slug': 'team-heretics-lec'},
 {'team_id': '101157821387445307',
  'name': 'Caps',
  'acronym': 'EU1',
  'slug': 'caps'},
 {'team_id': '110534724851488577',
  'name': 'Froggy Five',
  'acronym': 'FROG',
  'slug': 'froggy-five'},
 {'team_id': '109671201259007744',
  'name': 'Team Liquid Honda First',
  'acronym': 'TLF',
  'slug': 'team-liquid-honda-first'},
 {'team_id': '107423086908356081',
  'name': 'Nyyrikki',
  'acronym': 'NKI',
  'slug': 'nyyrikki'},
 {'team_id': '107582176093752697',
  'name': 'Hooked Esports',
  'acronym': 'HK',
  'slug': 'hooked-esports'},
 {'tea